In [1]:
#Multiple linear regression using tensorflow
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import time
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format



In [2]:
House_Price1=pd.read_csv("../input/data.csv")
House_Price1.tail()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
4595,2014-07-09 00:00:00,308166.7,3.0,1.8,1510,6360,1.0,0,0,4,1510,0,1954,1979,501 N 143rd St,Seattle,WA 98133,USA
4596,2014-07-09 00:00:00,534333.3,3.0,2.5,1460,7573,2.0,0,0,3,1460,0,1983,2009,14855 SE 10th Pl,Bellevue,WA 98007,USA
4597,2014-07-09 00:00:00,416904.2,3.0,2.5,3010,7014,2.0,0,0,3,3010,0,2009,0,759 Ilwaco Pl NE,Renton,WA 98059,USA
4598,2014-07-10 00:00:00,203400.0,4.0,2.0,2090,6630,1.0,0,0,3,1070,1020,1974,0,5148 S Creston St,Seattle,WA 98178,USA
4599,2014-07-10 00:00:00,220600.0,3.0,2.5,1490,8102,2.0,0,0,4,1490,0,1990,0,18717 SE 258th St,Covington,WA 98042,USA


In [3]:
#Check for null valuess
House_Price1.isnull().sum()

date            0
price           0
bedrooms        0
bathrooms       0
sqft_living     0
               ..
yr_renovated    0
street          0
city            0
statezip        0
country         0
Length: 18, dtype: int64

In [4]:
#Randomizing the data to make sure that no pathological ordering effects  the performance of Stochastic Gradient Descent.
House_Price2 = House_Price1.reindex(np.random.permutation(House_Price1.index))
House_Price2.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
3795,2014-07-01 00:00:00,808000.0,4.0,2.2,2500,8866,2.0,0,0,4,2500,0,1987,0,14586 NE 58th St,Bellevue,WA 98007,USA
3273,2014-06-24 00:00:00,395000.0,3.0,2.5,1600,1936,2.0,0,0,3,1600,0,2007,0,4517-4529 S Genesee St,Seattle,WA 98108,USA
202,2014-05-06 00:00:00,562000.0,5.0,2.2,3040,8111,2.0,0,0,3,3040,0,1984,0,17649 NE 129th Pl,Redmond,WA 98052,USA
1767,2014-06-03 00:00:00,440000.0,2.0,1.0,1230,6600,1.0,0,0,3,1130,100,1906,2014,3934 S Hudson St,Seattle,WA 98118,USA
1519,2014-05-28 00:00:00,499950.0,4.0,2.5,2798,4473,2.0,0,0,3,2798,0,2012,1912,19296-19336 102nd Ave SE,Renton,WA 98055,USA


In [5]:
#Examine the data
House_Price2.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
count,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0,4600.0
mean,551963.0,3.4,2.2,2139.3,14852.5,1.5,0.0,0.2,3.5,1827.3,312.1,1970.8,808.6
std,563834.7,0.9,0.8,963.2,35884.4,0.5,0.1,0.8,0.7,862.2,464.1,29.7,979.4
min,0.0,0.0,0.0,370.0,638.0,1.0,0.0,0.0,1.0,370.0,0.0,1900.0,0.0
25%,322875.0,3.0,1.8,1460.0,5000.8,1.0,0.0,0.0,3.0,1190.0,0.0,1951.0,0.0
50%,460943.5,3.0,2.2,1980.0,7683.0,1.5,0.0,0.0,3.0,1590.0,0.0,1976.0,0.0
75%,654962.5,4.0,2.5,2620.0,11001.2,2.0,0.0,0.0,4.0,2300.0,610.0,1997.0,1999.0
max,26590000.0,9.0,8.0,13540.0,1074218.0,3.5,1.0,4.0,5.0,9410.0,4820.0,2014.0,2014.0


In [6]:
# Define the input feature
House_Price_Feature=House_Price2[['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','sqft_above',
                           'sqft_basement']]

# Configure a numeric feature column for total_rooms.
House_Price_Feature_columns = [tf.feature_column.numeric_column("bedrooms"),tf.feature_column.numeric_column("bathrooms"),
                               tf.feature_column.numeric_column("sqft_living"),tf.feature_column.numeric_column("sqft_lot"),
                               tf.feature_column.numeric_column("floors"),tf.feature_column.numeric_column("waterfront"),
                               tf.feature_column.numeric_column("view"),tf.feature_column.numeric_column("condition"),
                               tf.feature_column.numeric_column("sqft_above"),tf.feature_column.numeric_column("sqft_basement")]
House_Price_Feature_columns

[NumericColumn(key='bedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='bathrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sqft_living', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sqft_lot', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='floors', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='waterfront', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='view', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='condition', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sqft_above', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sqft_basement', shape=(1,), default_value=None, dtype=tf.fl

In [7]:
# Define the label.
House_Price_Targets = House_Price2["price"]
House_Price_Targets

3795    808000.0
3273    395000.0
202     562000.0
1767    440000.0
1519    499950.0
          ...   
3608    516500.0
910     350000.0
482    1575000.0
3183    530000.0
2475    590000.0
Name: price, Length: 4600, dtype: float64

In [8]:
# Use gradient descent as the optimizer for training the model.
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=House_Price_Feature_columns,
    optimizer=my_optimizer
)
linear_regressor


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [9]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [10]:
_ = linear_regressor.train(
    input_fn = lambda:my_input_fn(House_Price_Feature, House_Price_Targets),
    steps=100
)

In [11]:
# Create an input function for predictions.
# Note: Since we're making just one prediction for each example, we don't 
# need to repeat or shuffle the data here.
prediction_input_fn =lambda: my_input_fn(House_Price_Feature, House_Price_Targets, num_epochs=1, shuffle=False)

# Call predict() on the linear_regressor to make predictions.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Format predictions as a NumPy array, so we can calculate error metrics.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
mean_squared_error = metrics.mean_squared_error(predictions, House_Price_Targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 622502709200.170
Root Mean Squared Error (on training data): 788988.409


In [12]:
min_house_value = House_Price2["price"].min()
max_house_value = House_Price2["price"].max()
min_max_difference = max_house_value - min_house_value

print("Min. Median House Value: %0.3f" % min_house_value)
print("Max. Median House Value: %0.3f" % max_house_value)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)


Min. Median House Value: 0.000
Max. Median House Value: 26590000.000
Difference between Min. and Max.: 26590000.000
Root Mean Squared Error: 788988.409
